In [26]:
# Importuri necesare
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

#df = pd.read_csv('../data/results/hotel_booking_cancellation_prediction_encoded_ordinal_deposit.csv')
df = pd.read_csv('../data/results/hotel_booking_cancellation_prediction_one_hot_deposit.csv')


In [29]:
# Separă feature-urile (toate coloanele în afară de target)

df = df.sample(frac=1, random_state=42).reset_index(drop=True)


df['arrival_date_month'] = pd.to_datetime(df['arrival_date']).dt.month

'''drop_columns = [
    'Index',                      # Index redundant
    'reservation_status_date',    # Scurgere de date (informație post-anulare)
    'reservation_status',         # Scurgere de date (corelație directă cu ținta)
    'reservation_status_Check-Out',  # Scurgere de date (derivat din reservation_status)
    'reservation_status_No-Show',    # Scurgere de date (derivat din reservation_status)
    'agent',                      # Înlocuit de is_agent
    'company',                    # Înlocuit de is_company
    'deposit_type',               # Înlocuit de is_non_refund/no_deposit
    'deposit_risk',               # Redundant cu noile variabile binare
    'stays_in_weeks_nights',      # Posibilă eroare de naming? (redundant cu stays_in_week_nights)
    'lead_time_group',            # Redundant cu lead_time
    'cancellation_risk_lead_time',# Redundant cu lead_time
    'is_children',                # Redundant cu children
    'is_babies',                  # Redundant cu babies
    'arrival_date',               # Informația e deja captată în alte coloane de dată
    'assigned_room_type_B',       # Cauzează scurgeri de date (cunoscută după check-in)
    'assigned_room_type_C',       # || 
    'assigned_room_type_D',       # ||
    'assigned_room_type_E',       # || -> Toate acestea sunt informații 
    'assigned_room_type_F',       # ||   disponibile doar DUPĂ ce rezervarea 
    'assigned_room_type_G',       # ||   a fost procesată (scurgere de date)
    'assigned_room_type_H',       # ||
    'assigned_room_type_I',       # ||
    'assigned_room_type_K'        # ||
]'''


X = df.drop(columns=['is_canceled', 'Index', 'arrival_date', 'stays_in_weeks_nights'], errors='ignore')

# Target-ul
y = df['is_canceled']

print(X.dtypes.value_counts())
df['arrival_date'] = pd.to_datetime(df['arrival_date'], errors='coerce')
print(X.select_dtypes(include='object').columns.tolist())

#X = X.dropna()
#y = y[X.index]
print(X.isnull().sum()[X.isnull().sum() > 0])


[col for col in df.columns if 'reservation_status' in col]
[col for col in df.columns if 'assigned_room_type' in col]
[col for col in df.columns if 'deposit_' in col]


print(df['is_canceled'].value_counts(normalize=True))

[col for col in X.columns if 'reservation_status' in col]
[col for col in X.columns if 'deposit_' in col]
[col for col in X.columns if 'assigned_room_type' in col]
[col for col in X.columns if 'reserved_room_type' in col]

X.columns


bool       53
int64      21
float64     2
int32       1
Name: count, dtype: int64
[]
Series([], dtype: int64)
is_canceled
0    0.618975
1    0.381025
Name: proportion, dtype: float64


Index(['lead_time', 'arrival_date_year', 'arrival_date_month',
       'arrival_date_week_number', 'arrival_date_day_of_month',
       'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children',
       'babies', 'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'booking_changes',
       'days_in_waiting_list', 'adr', 'required_car_parking_spaces',
       'total_of_special_requests', 'is_agent', 'is_company', 'is_children',
       'is_babies', 'room_changed', 'hotel_Resort Hotel', 'meal_FB', 'meal_HB',
       'meal_SC', 'country_Other', 'country_AUS', 'country_AUT', 'country_BEL',
       'country_BRA', 'country_CHE', 'country_CHN', 'country_CN',
       'country_DEU', 'country_DNK', 'country_ESP', 'country_FIN',
       'country_FRA', 'country_GBR', 'country_IRL', 'country_ISR',
       'country_ITA', 'country_NLD', 'country_NOR', 'country_POL',
       'country_PRT', 'country_ROU', 'country_RUS', 'country_SWE',
       'country_USA', 'countr

In [30]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Split date train/test (80%-20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creează scalerul și potrivește-l pe datele de train
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Transformă datele de test folosind același scaler
X_test_scaled = scaler.transform(X_test)

# Creează modelul
model = LogisticRegression(max_iter=15000)

# Antrenează modelul pe datele scalate
model.fit(X_train_scaled, y_train)

# Prezice pe datele scalate de test
y_pred = model.predict(X_test_scaled)

# Evaluare simplă
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.8209030948756976

Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.91      0.87      1248
           1       0.81      0.66      0.73       723

    accuracy                           0.82      1971
   macro avg       0.82      0.79      0.80      1971
weighted avg       0.82      0.82      0.82      1971


Confusion Matrix:
 [[1138  110]
 [ 243  480]]
